# TAL - Classification de dépêches d’agence avec NLTK

In [32]:
# importing modules
import nltk
from nltk.corpus import reuters
from nltk.corpus import stopwords 
import string
from nltk.stem import WordNetLemmatizer

In [35]:
lemmatizer = WordNetLemmatizer()

# Extract fileids from the reuters corpus
fileids = reuters.fileids()

# Get all the english stopwords
stop_words = set(stopwords.words('english')) 
dataset = []

# Loop through each file id and collect each files categories and tokenized words (w/o stopwords)
for file in fileids:
    words = reuters.words(file)
    #filtered_words = [lemmatizer.lemmatize(w.lower()) for w in words if not w.lower() in stop_words and w[0].isalnum()]
    dataset.append((words, reuters.categories(file)))

print(dataset[0])

(['asian', 'exporter', 'fear', 'damage', 'u', 'japan', 'rift', 'mounting', 'trade', 'friction', 'u', 'japan', 'raised', 'fear', 'among', 'many', 'asia', 'exporting', 'nation', 'row', 'could', 'inflict', 'far', 'reaching', 'economic', 'damage', 'businessmen', 'official', 'said', 'told', 'reuter', 'correspondent', 'asian', 'capital', 'u', 'move', 'japan', 'might', 'boost', 'protectionist', 'sentiment', 'u', 'lead', 'curb', 'american', 'import', 'product', 'exporter', 'said', 'conflict', 'would', 'hurt', 'long', 'run', 'short', 'term', 'tokyo', 'loss', 'might', 'gain', 'u', 'said', 'impose', '300', 'mln', 'dlrs', 'tariff', 'import', 'japanese', 'electronics', 'good', 'april', '17', 'retaliation', 'japan', 'alleged', 'failure', 'stick', 'pact', 'sell', 'semiconductor', 'world', 'market', 'cost', 'unofficial', 'japanese', 'estimate', 'put', 'impact', 'tariff', '10', 'billion', 'dlrs', 'spokesman', 'major', 'electronics', 'firm', 'said', 'would', 'virtually', 'halt', 'export', 'product', 'hi

> Note: We've noticed that in the tokenized words of the corpus, the word `U.S` is split into three tokens

## Classifieurs binaire

### Premier classifieur

### Deuxième classifieur

### Troisième classifieur

## Classifieur multiclasse